In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/billsWithHighConfidence.csv")


def clean_text(text):
    return re.sub(r"\s+", " ", text)


prototypes = {
    "Precedent": "This citation refers to a prior court decision or established case law that sets a legal standard for future rulings.",
    "Authority": "This citation invokes a binding legal source, such as a statute, regulation, or constitutional provision, to support an argument.",
    "Definition": "This citation is used to define or clarify a specific legal term or concept for accurate interpretation.",
    "Example": "This citation introduces a typical case or illustrates the subject by example.",
    "Exception": "This citation identifies a deviation from the general rule, highlighting a specific exception in the law.",
    "Amendment": "This citation indicates a modification or update to an existing law or regulation."
}


model = SentenceTransformer('all-MiniLM-L6-v2')


prototype_embeddings = {label: model.encode(text) for label, text in prototypes.items()}


def assign_low_confidence(context):
    if not isinstance(context, str) or not context.strip():
        return ""
    context = clean_text(context)
    context_emb = model.encode(context)
    best_label = None
    best_score = -1
    for label, proto_emb in prototype_embeddings.items():
        score = util.cos_sim(context_emb, proto_emb).item()
        if score > best_score:
            best_score = score
            best_label = label
    return best_label

# Apply to each row
df["low_confidence"] = df["context"].apply(assign_low_confidence)

# Save the result
df.to_csv("/content/drive/MyDrive/Colab Notebooks/billsWithConfidence_full.csv", index=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]